Removing 0 values from Temprature Data

In [1]:
import rasterio
import numpy as np

def process_geotiff(input_file, output_file):
    # Open the input TIFF file
    with rasterio.open(input_file) as src:
        # Read the data from the first band
        data = src.read(1)
        
        # Set negative and zero values to NaN
        data[data == -99] = np.nan
        
        # Update the metadata to handle NaN values correctly
        profile = src.profile
        profile.update(
            dtype=rasterio.float32,  # Ensure data type can handle NaN
            nodata=np.nan            # Define nodata value
        )
        
        # Write the modified data to a new TIFF file
        with rasterio.open(output_file, 'w', **profile) as dst:
            dst.write(data, 1)

# Define input and output file paths
input_file = 'UK_Avg_temp_Band1.tif'
output_file = 'UK_temp2.tif'

# Process the TIFF file
process_geotiff(input_file, output_file)

Removing 0 Values from DEM 

In [2]:
import rasterio
import numpy as np

def process_geotiff(input_file, output_file):
    # Open the input TIFF file
    with rasterio.open(input_file) as src:
        # Read the data from the first band
        data = src.read(1)
        
        # Set negative and zero values to NaN
        data[data == 0] = np.nan
        
        # Update the metadata to handle NaN values correctly
        profile = src.profile
        profile.update(
            dtype=rasterio.float32,  # Ensure data type can handle NaN
            nodata=np.nan            # Define nodata value
        )
        
        # Write the modified data to a new TIFF file
        with rasterio.open(output_file, 'w', **profile) as dst:
            dst.write(data, 1)

# Define input and output file paths
input_file = 'UK_DEM_merged.tif'
output_file = 'UK_DEM2.tif'

# Process the TIFF file
process_geotiff(input_file, output_file)

Finding Difference in the 2 masks 

In [3]:
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import reproject
import numpy as np

# Paths to the input GeoTIFF files
raster1_path = 'UK_DEM2.tif'
raster2_path = 'UK_temp2.tif'
output_path = 'UK_imputation_mask_30m.tif'

# Function to create a binary mask where data is present
def create_presence_mask(data, nodata_value):
    mask = np.ones_like(data, dtype=np.uint8)
    if nodata_value is not None and np.isnan(nodata_value):
        mask[np.isnan(data)] = 0
    elif nodata_value is not None:
        mask[data == nodata_value] = 0
    return mask

# Open the input rasters
with rasterio.open(raster1_path) as src1:
    data1 = src1.read(1)
    meta1 = src1.meta
    nodata1 = src1.nodata

with rasterio.open(raster2_path) as src2:
    data2 = src2.read(1)
    nodata2 = src2.nodata

    # Resample raster2 to match raster1 if dimensions differ
    if src1.shape != src2.shape:
       
        data2_resampled = np.empty(src1.shape, dtype=np.float32)
        reproject(
            source=data2,
            destination=data2_resampled,
            src_transform=src2.transform,
            src_crs=src2.crs,
            dst_transform=src1.transform,
            dst_crs=src1.crs,
            resampling=Resampling.bilinear
        )
    else:
        data2_resampled = data2

# Create presence masks for each raster
mask1 = create_presence_mask(data1, nodata1)
mask2 = create_presence_mask(data2_resampled, nodata2)

# Calculate the difference mask
difference_mask = mask1 & ~mask2

# Update metadata for the output raster
meta1.update(dtype=rasterio.uint8, nodata=0)

# Save the difference mask as a new GeoTIFF file
with rasterio.open(output_path, 'w', **meta1) as dst:
    dst.write(difference_mask, 1)

print("The difference raster has been created and saved.")

The difference raster has been created and saved.


Points of Temprature Data 

In [4]:
import rasterio
from shapely.geometry import Point
import geopandas as gpd
import numpy as np

# Path to your GeoTIFF file
input_file = 'UK_DEM2.tif'

# Lists to store point geometries, coordinates, and pixel values
points = []
lats = []
lons = []
values = []

# Open the GeoTIFF file
with rasterio.open(input_file) as src:
    # Get the number of rows and columns in the raster
    rows, cols = src.shape
    
    # Get the nodata value
    nodata_value = src.nodata

    # Read the entire raster data into memory
    data = src.read(1)

    # Iterate through each pixel in the raster
    for row in range(rows):
        for col in range(cols):
            # Read pixel value
            pixel_value = data[row, col]
            
            # Check if pixel value is not nodata and not NaN
            if pixel_value != nodata_value and not np.isnan(pixel_value):
                # Convert pixel coordinates to geographic coordinates (longitude, latitude)
                lon, lat = src.xy(row, col)

                # Create a point geometry for each valid pixel coordinate
                point = Point(lon, lat)
                points.append(point)
                lats.append(lat)
                lons.append(lon)
                values.append(pixel_value)

# Create a GeoDataFrame from the list of points
gdf = gpd.GeoDataFrame(geometry=points, crs=src.crs)

# Add latitude, longitude, and pixel values to the GeoDataFrame
gdf['latitude'] = lats
gdf['longitude'] = lons
gdf['value'] = values

# Save the GeoDataFrame as a shapefile
output_shapefile = 'UK_temp2.shp'
gdf.to_file(output_shapefile)

print(f"Shapefile saved successfully: {output_shapefile}")

Shapefile saved successfully: UK_temp2.shp



use trialruns code to now decide the file with most number of points available 